In [1]:
!pip install lightfm

  Using cached lightfm-1.16.tar.gz (310 kB)
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-win_amd64.whl size=430914 sha256=b9d19650df03cd1e2c421bab7191b61c117196798d9b7e806410471c9f186121
  Stored in directory: c:\users\roman\appdata\local\pip\cache\wheels\ec\bb\51\9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm


In [3]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from pathlib import Path

In [4]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

In [5]:
DATA_FOLDER = Path("../data/raw/")
ratings = pd.read_csv(DATA_FOLDER/"ratings.csv")
books = pd.read_csv(DATA_FOLDER/"books.csv")
tags = pd.read_csv(DATA_FOLDER/"tags_cleaned.csv")
book_tags = pd.read_csv(DATA_FOLDER/"book_tags.csv")


In [6]:
tags.head()

,tag_id,tag_name
0,509,19th-century
1,923,20th-century
2,941,21st-century
3,1499,abuse
4,1540,action


In [7]:
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [8]:
book_tags 

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716
...,...,...,...
999907,33288638,21303,7
999908,33288638,17271,7
999909,33288638,1126,7
999910,33288638,11478,7


In [9]:
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]

In [10]:
book_tags["id"] = book_tags.goodreads_book_id.apply(lambda book_id: mapper[book_id])

<ipython-input-10-aa7c749ce28e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_tags["id"] = book_tags.goodreads_book_id.apply(lambda book_id: mapper[book_id])


In [11]:
ratings_coo = sparse.coo_matrix((ratings.rating, (ratings.user_id, ratings.book_id)))

In [12]:
book_tags_coo = sparse.coo_matrix(([1]*len(book_tags), (book_tags.id, book_tags.tag_id)))

In [13]:
#число потоков нашего процессора
NUM_THREADS = 8 

#число параметров вектора 
NUM_COMPONENTS = 30 

#число эпох обучения
NUM_EPOCHS = 10 

In [ ]:
#Создаём модель
model = LightFM(learning_rate=0.05, loss='warp', no_components=NUM_COMPONENTS)
 
#Разбиваем наш датасет на обучающую и тестовую выборки
train,test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=None)

#Обучаем модель
model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS,item_features =book_tags_coo)

In [ ]:
#Тестируем нашу модель
prec_score = precision_at_k(
                     model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()
 
recall_at_k = recall_at_k(model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()

print(recall_at_k,prec_score)